In [ ]:
import tensorflow as tf
import sys

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.apps as app

The ini config file is not used in this tutorial, though it might be of reference value.

In [ ]:
# config = helper.ConfigParser(sys.argv, 'inference')

Data feeders for inference.

In [ ]:
reader_moving_image, reader_fixed_image, _, _ = helper.get_data_readers('./data/test/mr_images', './data/test/us_images')

Placeholders do not need to have the same minibatch size of those in training.

In [ ]:
ph_moving_image = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
ph_fixed_image = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])

Load the same 'local' network.

In [ ]:
reg_net = network.build_network(network_type='local',
                                minibatch_size=reader_moving_image.num_data,
                                image_moving=ph_moving_image,
                                image_fixed=ph_fixed_image)

Now populate the saved network weights to the loaded network, in this case, from ./data/model.ckpt.

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, './data/model.ckpt')

Feed the test images and save the predicted DDFs at ./data/.

In [ ]:
testFeed = {ph_moving_image: reader_moving_image.get_data(),
            ph_fixed_image: reader_fixed_image.get_data()}
ddf = sess.run(reg_net.ddf, feed_dict=testFeed)
helper.write_images(ddf, './data/', 'ddf')

Save the DDF-warped images at ./data/.

In [ ]:
warped_images = app.warp_volumes_by_ddf(reader_moving_image.get_data(), ddf)
helper.write_images(warped_images, './data/', 'warped_image')

Save the DDF-warped labels at ./data/.

In [ ]:
data_moving_label = helper.DataReader('./data/test/mr_labels').get_data(label_indices=[0])
warped_labels = app.warp_volumes_by_ddf(data_moving_label, ddf)
helper.write_images(warped_labels, './data/', 'warped_label')

Now the registration can be evaluated by comparing the warped moving labels with the corresponding fixed labels. 